# UKB Virus Regression and FDR

In [1]:
# Imports here.
import numpy as np
import pandas as pd
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf
import warnings
warnings.filterwarnings("ignore")

#Directory in Biowulf
os.chdir('/PATH/TO/UKB_Files')

# Loading Files

In [2]:
#Loading all the reference files
NDD_free = pd.read_csv('ALL_NDD_FREE_CONTROLS_AGE60PLUS.txt', delimiter='\t')
phenome = pd.read_csv('covariates_phenome_to_use.txt', delimiter='\t')
massive_ICD10 = pd.read_csv('massive_ICD10_ALL_table.txt', delimiter='\t', header = None)

In [3]:
#Loading all the disease files
AD = pd.read_csv("alzheimer_disease.txt", delimiter = '\t')
ALS = pd.read_csv('ALS.IDs', header = None) 
dementia = pd.read_csv('Dementia.IDs', header = None)
PD = pd.read_csv('parkinson_disease.txt', delimiter='\t')
vascular = pd.read_csv('Vascular.IDs', header = None)

In [4]:
#Getting MS IDs
#G35 is the ICD10 code for Multiple sclerosis
G35 = massive_ICD10.loc[massive_ICD10[1] == 'G35']
list_MS = G35[0]
MS = phenome[phenome['IID'].isin(list_MS)]
MS = MS.rename(columns = {'FID': 0})
#MS

# Adding Disease Column

In [5]:
#Pick your NDD here
NDD = AD
ndd = "AD"

In [6]:
#Add disease column to NDD free df
NDD_free[ndd] = 0

#Drop FID, Batch, and European
NDD_free = NDD_free.drop(columns = ['FID', 'BATCH', "EUROPEAN"])

#Rename ID column
NDD_free = NDD_free.rename(columns = {'IID': 'ID'})
print("Number of controls:", len(NDD_free))
#NDD_free

Number of controls: 96390


In [7]:
#Creating df of people with NDD
NDD = NDD.rename(columns = {'eid': 0})
NDD_list = list(NDD[0])
has_NDD = phenome[phenome['IID'].isin(NDD_list)]

#Only select Europeans
has_NDD = has_NDD[has_NDD['EUROPEAN'] == 1]

#Drop FID, Batch, and European
has_NDD = has_NDD.drop(columns = ['FID', 'BATCH', "EUROPEAN"])

#Rename ID column
has_NDD = has_NDD.rename(columns = {'IID': 'ID'})

#Add NDD column
has_NDD[ndd] = 1

print(f"Number of individuals with {ndd}:", len(has_NDD))

Number of individuals with AD: 2342


In [8]:
#Combine NDD_free and has_NDD
df = pd.concat([NDD_free, has_NDD])
df

,ID,BIRTH_YEAR,TOWNSEND,AGE_OF_RECRUIT,GENETIC_SEX,AD
0,1000012,1949,-4.901740,61,1,0
1,1000047,1943,-2.440140,65,0,0
2,1000068,1948,-4.377210,61,1,0
3,1000085,1944,-0.774111,65,0,0
4,1000094,1946,-1.702940,61,1,0
...,...,...,...,...,...,...
501957,6019584,1939,3.642590,69,0,1
502065,6020668,1941,-4.469670,68,0,1
502072,6020733,1940,-2.586190,69,0,1
502341,6023425,1948,-3.100560,60,1,1


# Adding ICD10 Codes to dataframe

In [9]:
#Number of codes
search_terms = pd.read_csv('UKB_search_terms.csv')
unique_codes = list(search_terms['coding'])
print("Unique codes:", len(unique_codes))

Unique codes: 95


In [10]:
#Finding Viral codes in ICD10 list
for code in unique_codes:
    viral_ICD10 = massive_ICD10[massive_ICD10[1] == code]
    viral_ICD10 = viral_ICD10.rename(columns = {0: 'ID', 1: "Code"})
    viral_ICD10 = list(viral_ICD10["ID"])
    df[code] = np.where(df['ID'].isin(viral_ICD10), 1, 0)

In [11]:
#Checking that ICD10 columns were added
df.head()

,ID,BIRTH_YEAR,TOWNSEND,AGE_OF_RECRUIT,GENETIC_SEX,AD,A080,A083,A084,A600,...,A492,G510,B012,B018,B019,B022,B023,B028,B029,G530
0,1000012,1949,-4.901740,61,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1000047,1943,-2.440140,65,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1000068,1948,-4.377210,61,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1000085,1944,-0.774111,65,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1000094,1946,-1.702940,61,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# Create lists for the regression
predictor_list = list(search_terms['coding'])
predictor_meaning = list(search_terms['meaning'])

# Regressions

In [13]:
# Now nothing left to do is run the regressions and call it a day. 
from statsmodels.stats.multitest import fdrcorrection
results = []

# for predictor in range(1, 10):
for predictor in range(len(predictor_list)):
  predictor_name = predictor_list[predictor]
  predictor_description = predictor_meaning[predictor]
  this_formula = ndd + "~ df['" + predictor_list[predictor] + "']" + " + AGE_OF_RECRUIT + TOWNSEND + GENETIC_SEX"
  fitted = sm.formula.glm(formula=this_formula, family=sm.families.Binomial(), data=df).fit()
  beta_coef  = fitted.params.loc["df['" + predictor_name + "']"]
  beta_se  = fitted.bse.loc["df['" + predictor_name + "']"]
  p_val = fitted.pvalues.loc["df['" + predictor_name + "']"]
  odds_ratio = np.exp(fitted.params.loc["df['" + predictor_name + "']"])
  conf = fitted.conf_int().loc["df['" + predictor_name + "']"]
  m5, m95 = np.exp(conf)
  n = sum(df[predictor_name])
  df2 = df[df[predictor_name]==1]
  n_pairs = sum(df2[ndd])  

  print(predictor_name, odds_ratio, m5, m95, p_val, n_pairs, n)
  results.append((ndd, predictor_name, predictor_description, odds_ratio, m5, m95, p_val, n_pairs, n))

output = pd.DataFrame(results, columns=('NDD','CODE', 'DESCRIPTION','odds_ratio', 'ci_min', "ci_max", 'P_VAL', "N_pairs", "N"))

A080 5.3079137491853784e-08 0.0 inf 0.9989304463954302 0 2
A083 5.839140492222715e-09 0.0 inf 0.9991093199219869 0 8
A084 2.978442283713354 1.7544468885405264 5.056361919733714 5.304423678221899e-05 15 203
A600 1.0000000000000413 1.000000000000037 1.0000000000000457 9.247159469078615e-77 0 0
A86 22.059257995012395 5.471083217394752 88.9423253046835 1.3676607510257822e-05 3 9
A879 6.339841866791054e-09 0.0 inf 0.998866415376507 0 13
B001 1.3900801193776806 0.18912504238567807 10.217170153218143 0.7462223418970593 1 31
B002 1.9171392805433713e-08 0.0 inf 0.9988053192297109 0 6
B003 1.0000000000000413 1.000000000000037 1.0000000000000457 9.247159469078615e-77 0 0
B004 20982884237.545437 0.0 inf 0.9989304625181011 1 1
B005 5.681052009896327e-09 0.0 inf 0.9987790189476798 0 15
B008 1.4823850027435405e-08 0.0 inf 0.9990146486402869 0 4
B009 6.156425076488354e-09 0.0 inf 0.9986998279828917 0 17
B07 1.6597055966345904 0.8162518376400794 3.3747215509667416 0.16174341190687802 8 202
B178 28.8580

In [14]:
#Only looking at codes that have at least 2 pairings
output = output[output['N_pairs'] > 1]
output

,NDD,CODE,DESCRIPTION,odds_ratio,ci_min,ci_max,P_VAL,N_pairs,N
2,AD,A084,"A08.4_Viral_intestinal_infection,_unspecified",2.978442e+00,1.754447,5.056362,0.000053,15,203
4,AD,A86,A86_Unspecified_viral_encephalitis,2.205926e+01,5.471083,88.942325,0.000014,3,9
13,AD,B07,B07_Viral_warts,1.659706e+00,0.816252,3.374722,0.161743,8,202
16,AD,B181,B18.1_Chronic_viral_hepatitis_B_without_delta-...,4.564049e+00,1.065022,19.558785,0.040874,2,23
17,AD,B182,B18.2_Chronic_viral_hepatitis_C,3.505782e+00,0.824887,14.899632,0.089282,2,26
28,AD,B348,B34.8_Other_viral_infections_of_unspecified_site,4.583600e+00,1.055772,19.899545,0.042112,2,19
29,AD,B349,"B34.9_Viral_infection,_unspecified",1.713725e+00,1.001279,2.933102,0.049458,14,344
30,AD,B978,B97.8_Other_viral_agents_as_the_cause_of_disea...,2.041642e+00,0.636954,6.544114,0.229753,3,59
33,AD,G933,G93.3_Postviral_fatigue_syndrome,1.218847e+11,0.000000,inf,0.998598,4,4
40,AD,Z225,Z22.5_Carrier_of_viral_hepatitis,2.883511e+00,0.887839,9.365023,0.078065,3,44


In [15]:
#Adding FDR Correction

#Sort P-values
output = output.sort_values(by = "P_VAL")

#Drop Nan-values
output = output.dropna()

#FDR Correction
rejected, p_corr = fdrcorrection(output['P_VAL'], is_sorted=True)
output['P_CORR'] = p_corr
output['REJECTED'] = rejected

In [16]:
#Save output
output.to_csv('/data/levineks/Virus/regression_results/' + ndd + "_virus_UKB_ALL.csv", index=False)

In [17]:
#Check results
df2 = pd.read_csv('/data/levineks/Virus/regression_results/' + ndd + "_virus_UKB_ALL.csv")
df2.head(25)

,NDD,CODE,DESCRIPTION,odds_ratio,ci_min,ci_max,P_VAL,N_pairs,N,P_CORR,REJECTED
0,AD,A86,A86_Unspecified_viral_encephalitis,2.205926e+01,5.471083,88.942325,0.000014,3,9,0.000246,True
1,AD,A084,"A08.4_Viral_intestinal_infection,_unspecified",2.978442e+00,1.754447,5.056362,0.000053,15,203,0.000477,True
2,AD,B169,B16.9_Acute_hepatitis_B_without_delta-agent_an...,5.583029e+00,1.280888,24.334844,0.022048,2,19,0.132288,False
3,AD,B181,B18.1_Chronic_viral_hepatitis_B_without_delta-...,4.564049e+00,1.065022,19.558785,0.040874,2,23,0.141336,False
4,AD,B348,B34.8_Other_viral_infections_of_unspecified_site,4.583600e+00,1.055772,19.899545,0.042112,2,19,0.141336,False
5,AD,B349,"B34.9_Viral_infection,_unspecified",1.713725e+00,1.001279,2.933102,0.049458,14,344,0.141336,False
6,AD,B977,B97.7_Papillomavirus_as_the_cause_of_diseases_...,4.132548e+00,0.970283,17.601012,0.054964,2,25,0.141336,False
7,AD,Z225,Z22.5_Carrier_of_viral_hepatitis,2.883511e+00,0.887839,9.365023,0.078065,3,44,0.175646,False
8,AD,B182,B18.2_Chronic_viral_hepatitis_C,3.505782e+00,0.824887,14.899632,0.089282,2,26,0.178564,False
9,AD,B07,B07_Viral_warts,1.659706e+00,0.816252,3.374722,0.161743,8,202,0.284378,False
